In [1]:
!pip install gdown

In [3]:
# MFCC
!gdown --id 1AaP3PjDO_I1By-bGxe4a6pgg5I1PnZFM
!gdown --id 1IObT-XbciKO2GIrK6l3ZTqdfZOqck9F7
!gdown --id 1eS63BOtWKM6j94xZIHN265q0Dvd3N_LY

/opt/conda/lib/python3.10/site-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From (original): https://drive.google.com/uc?id=1AaP3PjDO_I1By-bGxe4a6pgg5I1PnZFM
From (redirected): https://drive.google.com/uc?id=1AaP3PjDO_I1By-bGxe4a6pgg5I1PnZFM&confirm=t&uuid=3343cde5-4f63-42ab-83cd-378db877311a
To: /kaggle/working/bee.npy
100%|█████████████████████████████████████████| 152M/152M [00:00<00:00, 176MB/s]
/opt/conda/lib/python3.10/site-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1IObT-XbciKO2GIrK6l3ZTqdfZOqck9F7
To: /kaggle/working/nobee.npy
100%|██████████████████████████████████████| 96.3M/96.3M [00:01<00:00, 57.7MB/s]
/opt/conda/lib/pyth

In [2]:
#STFT
!gdown 11xTOZgd8hUB-rGnwVBlW4uuWlHsAxyGr
!gdown 1e4OsgZXydshBZwSNdr4xpp9sYYF9p5P9
!gdown 17q829JKnZM6GjqEKTSshd-4AlKPO3aJd

Downloading...
From (original): https://drive.google.com/uc?id=11xTOZgd8hUB-rGnwVBlW4uuWlHsAxyGr
From (redirected): https://drive.google.com/uc?id=11xTOZgd8hUB-rGnwVBlW4uuWlHsAxyGr&confirm=t&uuid=96b97db8-c9b1-497e-b123-a443e078834f
To: /kaggle/working/no_queen.npy
100%|████████████████████████████████████████| 188M/188M [00:04<00:00, 39.7MB/s]
Downloading...
From (original): https://drive.google.com/uc?id=1e4OsgZXydshBZwSNdr4xpp9sYYF9p5P9
From (redirected): https://drive.google.com/uc?id=1e4OsgZXydshBZwSNdr4xpp9sYYF9p5P9&confirm=t&uuid=5f392058-4883-42f0-bdde-7a06841f7004
To: /kaggle/working/no_bee.npy
100%|████████████████████████████████████████| 134M/134M [00:03<00:00, 40.1MB/s]
Downloading...
From (original): https://drive.google.com/uc?id=17q829JKnZM6GjqEKTSshd-4AlKPO3aJd
From (redirected): https://drive.google.com/uc?id=17q829JKnZM6GjqEKTSshd-4AlKPO3aJd&confirm=t&uuid=adbc5e4b-8d11-4ccd-9c3a-25038bc07802
To: /kaggle/working/bee.npy
100%|████████████████████████████████████████| 

In [4]:
!gdown 163T-i5D6h1M69RSTrMEMg_00HX4p-vWe

/opt/conda/lib/python3.10/site-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=163T-i5D6h1M69RSTrMEMg_00HX4p-vWe
To: /kaggle/working/RegNet_MFCC_3labels_89_92.pth
100%|██████████████████████████████████████| 15.9M/15.9M [00:00<00:00, 35.7MB/s]


In [38]:
!gdown 17vUEuzOMXWCQmYBd4dotr1orQSPE3ivP

Downloading...
From: https://drive.google.com/uc?id=17vUEuzOMXWCQmYBd4dotr1orQSPE3ivP
To: /kaggle/working/RegNet_STFT_3labels_78.25.pth
100%|██████████████████████████████████████| 15.9M/15.9M [00:00<00:00, 43.0MB/s]


In [3]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms, datasets, models
from torch.utils.data import DataLoader, Dataset
import numpy as np
from tqdm import tqdm

In [4]:
bee = np.load('/kaggle/working/bee.npy')
nobee = np.load('/kaggle/working/no_bee.npy')
noqueen = np.load('/kaggle/working/no_queen.npy')

In [5]:
labels_bee = np.ones(len(bee)) * 1  
labels_nobee = np.ones(len(nobee)) * 0  
labels_noqueen = np.ones(len(noqueen)) * 2  

In [6]:
data = np.concatenate((bee, nobee, noqueen), axis=0)
labels = np.concatenate((labels_bee, labels_nobee, labels_noqueen), axis=0)

df = pd.DataFrame({'data': list(data), 'labels': labels})
df = df.sample(frac=1).reset_index(drop=True) 

In [7]:
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42, stratify=df['labels'])
train_df, val_df = train_test_split(train_df, test_size=0.15, random_state=42, stratify=train_df['labels'])

x_train = np.array(train_df['data'].tolist())
y_train = np.array(train_df['labels'].tolist())

x_val = np.array(val_df['data'].tolist())
y_val = np.array(val_df['labels'].tolist())

x_test = np.array(test_df['data'].tolist())
y_test = np.array(test_df['labels'].tolist())

In [8]:
x_train = x_train[..., np.newaxis]
x_val = x_val[..., np.newaxis]
x_test = x_test[..., np.newaxis]

In [9]:
x_train.shape

(9378, 111, 87, 1)

In [10]:
class BeeDataset(Dataset):
    def __init__(self, images, labels, transform=None):
        self.images = images
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image = self.images[idx]
        label = self.labels[idx]
        if self.transform:
            image = self.transform(image)
        return image, label

In [11]:
transform = transforms.Compose([
    #transforms.ToPILImage(),  # Convert numpy array to PIL Image
    #transforms.Resize((224, 224)), 
    transforms.ToTensor()
])

In [12]:
train_dataset = BeeDataset(x_train, y_train, transform=transform)
val_dataset = BeeDataset(x_val, y_val, transform=transform)
test_dataset = BeeDataset(x_test, y_test, transform=transform)

In [13]:
train_dataloader = DataLoader(train_dataset, batch_size=512, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=512, shuffle=False)
test_dataloader = DataLoader(test_dataset, batch_size=512, shuffle=False)

In [14]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [15]:
seed = 42
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
# Ensure deterministic behavior
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [38]:
model = models.regnet_y_400mf(weights='IMAGENET1K_V1')

model.stem[0] = nn.Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)

num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 3)

#weight_path = '/kaggle/working/best_model.pth'
#state_dict = torch.load(weight_path)
#model.load_state_dict(state_dict)

model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

In [39]:
# Training and validation loop
num_epochs = 50
train_accs = []
val_accs = []
train_losses = []
val_losses = []
best_val_accuracy = 0
best_val_loss = float('inf')

for epoch in range(num_epochs):
    # Training
    model.train()
    train_loss = 0.0
    train_corrects = 0

    for train_images, train_labels in tqdm(train_dataloader, desc=f"Epoch {epoch+1}"):
        train_images = train_images.to(device)
        train_labels = train_labels.long().to(device)
        
        optimizer.zero_grad()

        outputs = model(train_images)
        _, preds = torch.max(outputs, 1)
        loss = criterion(outputs, train_labels)
        
        loss.backward()
        optimizer.step()

        train_loss += loss.item() * train_images.size(0)
        train_corrects += torch.sum(preds == train_labels.data)

    train_loss /= len(train_dataloader.dataset)
    train_acc = train_corrects.double() / len(train_dataloader.dataset)
    train_losses.append(train_loss)
    train_accs.append(train_acc)

    # Validation
    model.eval()
    val_loss = 0.0
    val_corrects = 0
    with torch.no_grad():
        for val_images, val_labels in tqdm(val_dataloader, desc=f"Epoch {epoch+1} Validation"):
            val_images = val_images.to(device)
            val_labels = val_labels.long().to(device)
            
            val_outputs = model(val_images)
            _, preds = torch.max(val_outputs, 1)
            loss = criterion(val_outputs, val_labels)
            
            val_loss += loss.item() * val_images.size(0)
            val_corrects += torch.sum(preds == val_labels.data)

    val_loss /= len(val_dataloader.dataset)
    val_acc = val_corrects.double() / len(val_dataloader.dataset)
    val_losses.append(val_loss)
    val_accs.append(val_acc)

    print(f"Epoch {epoch+1}, Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}, Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}")

   
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save(model.state_dict(), 'best_model.pth')

Epoch 1 Validation: 100%|██████████| 4/4 [00:00<00:00,  4.30it/s]


Epoch 1, Train Loss: 0.8048, Train Acc: 0.6448, Val Loss: 0.6439, Val Acc: 0.7384


Epoch 2 Validation: 100%|██████████| 4/4 [00:00<00:00,  4.30it/s]


Epoch 2, Train Loss: 0.5212, Train Acc: 0.7882, Val Loss: 0.5472, Val Acc: 0.7716


Epoch 3 Validation: 100%|██████████| 4/4 [00:00<00:00,  4.31it/s]


...

Epoch 48, Train Loss: 0.0004, Train Acc: 1.0000, Val Loss: 1.1882, Val Acc: 0.7891


Epoch 49 Validation: 100%|██████████| 4/4 [00:00<00:00,  4.31it/s]


Epoch 49, Train Loss: 0.0005, Train Acc: 1.0000, Val Loss: 1.2057, Val Acc: 0.7879


Epoch 50 Validation: 100%|██████████| 4/4 [00:00<00:00,  4.31it/s]

Epoch 50, Train Loss: 0.0007, Train Acc: 0.9999, Val Loss: 1.2075, Val Acc: 0.7861


In [37]:
# Load the best model
best_model = model
best_model.load_state_dict(torch.load('best_model.pth'))

# Predict on the test set
best_model.eval()
test_predictions = []
true_labels = []
with torch.no_grad():
    for test_images, test_labels in tqdm(test_dataloader, desc="Test Set"):
        test_images = test_images.to(device)
        test_labels = test_labels.long().to(device)
            
        
        outputs = best_model(test_images)
        _, preds = torch.max(outputs, 1)
        
        test_predictions.extend(preds.cpu().numpy())
        true_labels.extend(test_labels.cpu().numpy())

# Evaluate the model on the test set
num_correct = sum([1 for i in range(len(test_predictions)) if test_predictions[i] == true_labels[i]])
test_accuracy = num_correct / len(test_predictions)

print(f'Test Accuracy: {test_accuracy * 100:.2f}%')

Test Set: 100%|██████████| 6/6 [00:01<00:00,  4.05it/s]

Test Accuracy: 71.22%
